In [1]:
import gensim.models.keyedvectors as word2vec
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim.models.fasttext as ft
import time
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import os
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import re
from sklearn.linear_model import LogisticRegression

In [2]:
CLEANED_NAMES_FILE = "../Data/category_cleaned_names.csv"

In [3]:
ft_wiki_model = word2vec.KeyedVectors.load_word2vec_format('../Data/wiki-news-300d-1M-subword.vec')



In [4]:
ft_wiki_vocab = [key.lower() for key in ft_wiki_model.vocab.keys()]

In [5]:
w2v_model = word2vec.KeyedVectors.load_word2vec_format('../Data/GoogleNews-vectors-negative300.bin', binary=True)
w2v_vocab = [key.lower() for key in w2v_model.vocab.keys()]

In [6]:
# glove_model_load = glove2word2vec("../Data/glove.840B.300d.txt", "../Data/gensim_glove_vectors.txt")
glove_model = word2vec.KeyedVectors.load_word2vec_format("../Data/gensim_glove_vectors.txt")


In [7]:
glove_vocab = [key.lower() for key in glove_model.vocab.keys()]

In [4]:
# ft_wiki_model = model = word2vec.KeyedVectors.load_word2vec_format('wiki.en.vec')
ft_cc_model = ft.load_facebook_model('crawl-300d-2M-subword.bin', encoding='utf-8')

# w2v_model = word2vec.KeyedVectors.load_word2vec_format('../Data/GoogleNews-vectors-negative300.bin', binary=True)
# w2v_vocab = [key.lower() for key in w2v_model.vocab.keys()]
# #https://datascience.stackexchange.com/questions/20071/how-do-i-load-fasttext-pretrained-model-with-gensim


In [27]:
ft_cc_model.wv.get_vector('fwefjwefijweoifjweoifweoifhwoie')

array([-6.89541129e-03,  3.49359168e-03,  2.79926397e-02,  2.50662607e-03,
       -1.41079882e-02,  2.09858734e-02, -2.97399182e-02,  1.34436108e-04,
        3.75853404e-02, -2.57807355e-02, -1.20127508e-02,  5.65120159e-03,
       -9.68236011e-03, -9.13783908e-04, -6.95809955e-03, -7.34106591e-03,
        1.12294098e-02,  3.56433145e-03,  1.22618279e-03,  2.34625805e-02,
       -4.65206541e-02, -3.41409841e-03,  5.39731048e-03,  9.66513157e-03,
       -5.00211716e-02,  5.08092577e-03, -5.88433705e-02,  4.85401368e-03,
        2.53504026e-03, -6.86202385e-03, -1.25101358e-02,  4.13493346e-03,
        1.81843005e-02, -1.27956774e-02,  6.41871057e-03,  6.22030301e-03,
        4.45850380e-03,  1.40668906e-03,  1.94280292e-03, -2.93512247e-03,
       -3.84467631e-03, -7.19417408e-02,  7.40049360e-03, -2.27805180e-03,
       -8.38536303e-03,  4.10385691e-02, -1.39689734e-02,  1.27894664e-02,
       -4.64180596e-02, -2.81508989e-03, -5.68726240e-03, -7.04045407e-03,
        5.84607525e-03,  

In [8]:
def preprocess(category):
    category = category[5:]
    category = re.sub(",|'s|& |/", ' ', category)
    category = re.sub("  ", " ", category )
    return category.lower()

In [9]:
def get_brand_categories(brands, concise=False):
    df_brands_category = pd.read_csv('../Data/brand_names_coded.csv')
    df_brands_no_dup = df_brands_category.drop_duplicates(subset='BRAND')
    if concise:
        cleaned_name_map = pd.read_csv(CLEANED_NAMES_FILE).set_index('Original').to_dict()['Concise']
    else:
        cleaned_name_map = pd.read_csv(CLEANED_NAMES_FILE).set_index('Original').to_dict()['Corrected']
    valid_df = df_brands_no_dup[df_brands_no_dup['BRAND'].isin(brands)]
    brand_category_mapping = valid_df.set_index('BRAND').to_dict()['CATEGORY']
    brand_category_cleaned = {brand: preprocess(cleaned_name_map[category]) for brand, category in brand_category_mapping.items()}
#     brand_category_mapping = {brand: preprocess(category) for brand, category in brand_category_cleaned.items()}
    return brand_category_cleaned, list(set(brand_category_cleaned.values()))

In [10]:
def get_category_vectors(model, model_vocab, category):
    words = category.split(" ")
    cat_vecs = np.array([])
    for word in words:
        if word != '':
#             index = model_vocab.index(word)
            try:
                word_vec = model.get_vector(word)
                if cat_vecs.shape[0] == 0:
                    cat_vecs = word_vec
                else:
                    cat_vecs = np.vstack([cat_vecs, word_vec])
            except KeyError:
                print(word)
                try:
                    word_vec = model.get_vector(w2v_model.index2entity[w2v_vocab.index(word)])
                    if cat_vecs.shape[0] == 0:
                        cat_vecs = word_vec
                    else:
                        cat_vecs = np.vstack([cat_vecs, word_vec])
                except:
                    word_vec = np.mean([model.get_vector('athletic'),model.get_vector('clothing')], axis=0)
                    print("NEW VEC")
                    print(word_vec.shape)
                cat_vecs = np.vstack([cat_vecs, word_vec])

        else:
            print(category)
            print("IT SLIPPED THROUGH")
#     print(np.mean(cat_vecs, axis=0).shape)
#     print(cat_vecs)
    cat_vecs = cat_vecs.reshape(-1, 300)
    return np.mean(cat_vecs, axis=0)



In [11]:
def get_simple_alignment(vectors, gender_subspace):
    gen_comp = cosine_similarity(vectors, gender_subspace)
#     if gen_comp < 0:
#         return 'M'
#     else:
#         return 'F'
    return gen_comp

In [12]:
def get_gender_axis(model):
    she_vec = model.get_vector('she')
    he_vec = model.get_vector('he')
    she_vec = she_vec.reshape(1, -1)
    he_vec = he_vec.reshape(1, -1)
    he_she_axis = normalize(np.subtract(she_vec, he_vec))
    return he_she_axis
    

In [13]:
def get_word_vectors(model, model_vocab, brands):
    vecs = np.array([])
    num_capitalized = 0
    for brand in tqdm(brands):
        try:
            word_vec = model.get_vector(brand.capitalize())
            vecs = np.append(vecs, word_vec.reshape(1, -1))
            num_capitalized +=1
        except KeyError:
            try:
                index = model_vocab.index(brand.lower())
            except ValueError:
                vecs = np.append(vecs, np.zeros(300).reshape(1, 300))
#                 word_vec = model.get_vector(brand)
#                 vecs = np.append(vecs, word_vec.reshape(1, -1))
                continue
            word_vec = model.get_vector(model.index2entity[index])
            vecs = np.append(vecs, word_vec.reshape(1, -1))
    print(f"NUM CAPITALIZED: {num_capitalized}/{len(brands)} = {np.round(num_capitalized/len(brands), 3)}")
    print(vecs.reshape(-1, 300).shape)
    return vecs.reshape(-1, 300)
        


In [14]:
def get_residual_vecs(vecs, category_vec_map, brand_category_mapping, brands):
    resid_vecs = []
    for i in range(len(brands)):
        resid_vecs = np.append(resid_vecs, np.subtract(vecs[i], category_vec_map[brand_category_mapping[brands[i]]]))
    print("RESIDUAL VECTORS SHAPE")
    print(resid_vecs.reshape(-1, 300).shape)
    return resid_vecs.reshape(-1, 300)

In [15]:
def get_ss_values(valid_brands):
    ratio_df_human = pd.read_csv('../Data/human_names_coded.csv')
    ratio_df_human = ratio_df_human.drop(["dataID"], axis=1)
    ratio_df_human = ratio_df_human.drop(["Name"], axis=1)
    ratio_x = ratio_df_human.drop(['Gender', 'is_unique'], axis=1).as_matrix()[:, 1:]
    ratio_y = ratio_df_human[['Gender']].as_matrix().ravel()
    ratio_model_LR = LogisticRegression()
    ratio_model_LR.fit(ratio_x, ratio_y)
    ratio_df_brands = pd.read_csv('../Data/brand_names_coded_no_duplicates.csv')
    ratio_maps = {}
    for i in range(len(ratio_df_brands.index)):
        feature_vec = ratio_df_brands.iloc[i][4:].as_matrix().ravel().reshape((1, -1))
        x = ratio_model_LR.predict_proba(feature_vec)
#         print(ratio_df_brands.iloc[i][2], x)
        ratio_maps[ratio_df_brands.iloc[i][2]] = x[0][1]
    valid_alignments = []
    for brand in valid_brands:
        valid_alignments.append(ratio_maps[brand])
    return valid_alignments
        

In [17]:
all_models =  [w2v_model, ft_wiki_model, glove_model] # 
all_vocabs =  [w2v_vocab, ft_wiki_vocab, glove_vocab] #w2v_vocab,
titles = ['Word2Vec', 'Wikipedia', 'Glove']
# all_models = [ft_cc_model.wv]
# all_vocabs =  [ft_cc_model.wv.vocab]
# titles = ['CommonCrawl']
alignment_df = pd.read_csv('../Data/alignment_scores.csv')
brands = alignment_df['brand'].tolist()


brand_category_mapping, categories = get_brand_categories(brands)
categories = list(categories)

    
for i in range(len(all_models)):
    
    curr_title = titles[i]
    curr_vocab = all_vocabs[i]
    curr_model = all_models[i]
    print(f"MODEL: {curr_title}")
    
    he_she_axis = get_gender_axis(curr_model)
    print(f"Getting word vectors")
    vecs = get_word_vectors(curr_model, curr_vocab, brands)
    
    alignment = get_simple_alignment(vecs, he_she_axis)
    print(alignment.shape)
    alignment_df[f'{curr_title}_alignment'] = alignment
    
    category_vec_map = {}
    for category in categories:
        new_vec = get_category_vectors(curr_model, curr_vocab, category)
        category_vec_map[category] = new_vec
    residual_vecs = get_residual_vecs(vecs, category_vec_map, brand_category_mapping, brands)
    
    resid_alignment = get_simple_alignment(residual_vecs, he_she_axis)
    alignment_df[f'{curr_title}_resid_alignment'] = resid_alignment
    

alignment_df.to_csv('../Data/all_models_alignment_27_08_v3.csv')
    
    
    
    
    
    

    

MODEL: Word2Vec
Getting word vectors



NUM CAPITALIZED: 1298/1471 = 0.882
(1471, 300)
(1471, 1)
RESIDUAL VECTORS SHAPE
(1471, 300)
MODEL: Wikipedia
Getting word vectors



NUM CAPITALIZED: 1127/1471 = 0.766
(1471, 300)
(1471, 1)
athleticwear
NEW VEC
(300,)
RESIDUAL VECTORS SHAPE
(1471, 300)
MODEL: Glove
Getting word vectors



NUM CAPITALIZED: 1370/1471 = 0.931
(1471, 300)
(1471, 1)
athleticwear
NEW VEC
(300,)
RESIDUAL VECTORS SHAPE
(1471, 300)


In [21]:
alignment_df = alignment_df.replace(0, np.nan).dropna()
alignment_df.to_csv('../Data/all_models_alignment_27_08_v3.csv')

In [38]:
from scipy.stats import pearsonr

In [22]:
# REPEAT CODE WITH FASTTEXT
df_models = pd.read_csv('../Data/all_models_alignment_27_08.csv')
df_models = df_models[df_models.columns[2:]]
pearsonr(df_models['Glove_alignment'], df_models['Word2Vec_alignment'])
pearsonr(df_models['Wikipedia_alignment'], df_models['Word2Vec_alignment'])
pearsonr(df_models['Glove_alignment'], df_models['Wikipedia_alignment'])

(0.33462343571387154, 8.138914780519281e-40)

In [23]:
df_models_trim =df_models.replace(0, np.nan).dropna()
print(pearsonr(df_models_trim['Glove_alignment'], df_models_trim['Word2Vec_alignment']))
print(pearsonr(df_models_trim['Wikipedia_alignment'], df_models_trim['Word2Vec_alignment']))
print(pearsonr(df_models_trim['Glove_alignment'], df_models_trim['Wikipedia_alignment']))
print(pearsonr(df_models_trim['Glove_resid_alignment'], df_models_trim['Word2Vec_resid_alignment']))
print(pearsonr(df_models_trim['Wikipedia_resid_alignment'], df_models_trim['Word2Vec_resid_alignment']))
print(pearsonr(df_models_trim['Glove_resid_alignment'], df_models_trim['Wikipedia_resid_alignment']))

(0.6712605153522405, 3.377764922892809e-166)
(0.4112802530312509, 9.867871380647285e-53)
(0.44779638845765096, 2.622405221877182e-63)
(0.6349455464765613, 1.6926896111528896e-143)
(0.5172240051202465, 2.4567665029618766e-87)
(0.4339555372724522, 3.807155025282788e-59)


In [20]:
df_models_trim['ss_alignment'] = get_ss_values(df_models_trim['brand'])

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

...

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

In [47]:
web_df = pd.read_csv('../Data/filtered_web_brands.csv', header=0)[['Name', 'Category', 'Overall Prediction', 'Gender in Photos Ratio (M:F)']]
web_df['mal'] = web_df['Gender in Photos Ratio (M:F)'].apply(lambda x: int(x.split(":")[0]))
web_df['fal'] = web_df['Gender in Photos Ratio (M:F)'].apply(lambda x: int(x.split(":")[1]))
# ratio_web_df['alignment'] = ratio_web_df['fal'] / ratio_web_df['mal']
web_df['alignment'] = (web_df['fal'] - web_df['mal']) / web_df[['fal', 'mal']].max(axis=1)
web_df['total'] = web_df['fal'] + web_df['mal']
web_df = web_df[web_df['total'] > 2]
sort_web_df = web_df.sort_values(by='Name')
sort_models_df = df_models_trim[df_models_trim['brand'].isin(sort_web_df['Name'].tolist())].sort_values(by='brand')
sort_web_df = sort_web_df[sort_web_df['Name'].isin(sort_models_df['brand'].tolist())]

In [48]:
len(web_df)

278

In [49]:
base_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Word2Vec_alignment'])
bert_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Wikipedia_alignment'])
corp_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Glove_alignment'])
category_corp_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Word2Vec_resid_alignment'])
residual_bert_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Wikipedia_resid_alignment'])
residual_corp_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Glove_resid_alignment'])
ss_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['ss_alignment'])

In [50]:
len(sort_web_df)

278

In [51]:
print(f"Correlation of Image Gender Ratio and Raw Word2Vec Embeddings: {np.round(base_correlation[0], 4)}, p-value: {np.round(base_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Raw Fasttext (Wikipedia) Embeddings: {np.round(bert_correlation[0], 4)}, p-value: {np.round(bert_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Raw Glove Embeddings: {np.round(corp_correlation[0], 4)}, p-value: {np.round(corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Word2Vec Embeddings: {np.round(category_corp_correlation[0], 4)}, p-value: {np.round(category_corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Fasttest (Wikipedia) Embeddings: {np.round(residual_bert_correlation[0], 4)}, p-value: {np.round(residual_bert_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Glove Embeddings: {np.round(residual_corp_correlation[0], 4)}, p-value: {np.round(residual_corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Sound Symbolism: {np.round(ss_correlation[0], 4)}, p-value: {np.round(ss_correlation[1], 4)}")


Correlation of Image Gender Ratio and Raw Word2Vec Embeddings: 0.4101, p-value: 0.0
Correlation of Image Gender Ratio and Raw Fasttext (Wikipedia) Embeddings: 0.2589, p-value: 0.0
Correlation of Image Gender Ratio and Raw Glove Embeddings: 0.4233, p-value: 0.0
Correlation of Image Gender Ratio and Residual Word2Vec Embeddings: 0.0739, p-value: 0.2191
Correlation of Image Gender Ratio and Residual Fasttest (Wikipedia) Embeddings: 0.0749, p-value: 0.2132
Correlation of Image Gender Ratio and Residual Glove Embeddings: -0.0374, p-value: 0.5349
Correlation of Image Gender Ratio and Sound Symbolism: 0.1715, p-value: 0.0041


In [52]:
titles = ['Raw W2V', 'Residual W2V', 'Raw FastText', 'Residual FastText', 'Raw Glove',   'Residual Glove', 'Sound Symbolism']
columns = sort_models_df.columns[2:].tolist()
results_df = {'Model': [], 'Correlation': [], 'P-value': []}
for i in range(len(titles)):
    corr, p = pearsonr(sort_web_df['alignment'], sort_models_df[columns[i]])
    results_df['Model'].append(titles[i])
    results_df['Correlation'].append(corr)
    results_df['P-value'].append(np.round(p, 4))

pd.DataFrame(results_df)

    
    

,Model,Correlation,P-value
0,Raw W2V,0.410076,0.0000
1,Residual W2V,0.073946,0.2191
2,Raw FastText,0.258918,0.0000
3,Residual FastText,0.074885,0.2132
4,Raw Glove,0.423347,0.0000
5,Residual Glove,-0.037375,0.5349
6,Sound Symbolism,0.171474,0.0041


In [99]:
sort_web_df['Name'].tolist() == sort_models_df['brand'].tolist()

True

# OLD FOR REFERENCE

In [100]:
base_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Word2Vec_alignment'])
bert_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Wikipedia_alignment'])
corp_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Glove_alignment'])
category_corp_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Word2Vec_resid_alignment'])
residual_bert_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Wikipedia_resid_alignment'])
residual_corp_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Glove_resid_alignment'])
ss_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['ss_alignment'])

In [102]:
print(f"Correlation of Image Gender Ratio and Raw Word2Vec Embeddings: {np.round(base_correlation[0], 4)}, p-value: {np.round(base_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Raw Fasttext (Wikipedia) Embeddings: {np.round(bert_correlation[0], 4)}, p-value: {np.round(bert_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Raw Glove Embeddings: {np.round(corp_correlation[0], 4)}, p-value: {np.round(corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Word2Vec Embeddings: {np.round(category_corp_correlation[0], 4)}, p-value: {np.round(category_corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Fasttest (Wikipedia) Embeddings: {np.round(residual_bert_correlation[0], 4)}, p-value: {np.round(residual_bert_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Glove Embeddings: {np.round(residual_corp_correlation[0], 4)}, p-value: {np.round(residual_corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Sound Symbolism: {np.round(ss_correlation[0], 4)}, p-value: {np.round(ss_correlation[1], 4)}")


Correlation of Image Gender Ratio and Raw Word2Vec Embeddings: 0.4357, p-value: 0.0001
Correlation of Image Gender Ratio and Raw Fasttext (Wikipedia) Embeddings: 0.341, p-value: 0.0023
Correlation of Image Gender Ratio and Raw Glove Embeddings: 0.4952, p-value: 0.0
Correlation of Image Gender Ratio and Residual Word2Vec Embeddings: 0.3195, p-value: 0.0043
Correlation of Image Gender Ratio and Residual Fasttest (Wikipedia) Embeddings: 0.368, p-value: 0.0009
Correlation of Image Gender Ratio and Residual Glove Embeddings: 0.1837, p-value: 0.1073
Correlation of Image Gender Ratio and Sound Symbolism: 0.1476, p-value: 0.1972


# CATEGORY ALIGNMENT SECTION

In [70]:
1471/72

20.430555555555557

In [23]:
all_models =  [w2v_model, ft_wiki_model, glove_model] # 
all_vocabs =  [w2v_vocab, ft_wiki_vocab, glove_vocab] #w2v_vocab,
titles = ['Word2Vec', 'Wikipedia', 'Glove']
# all_models = [ft_cc_model.wv]
# all_vocabs =  [ft_cc_model.wv.vocab]
# titles = ['CommonCrawl']
alignment_df = pd.read_csv('../Data/alignment_scores.csv')
brands = alignment_df['brand'].tolist()


brand_category_mapping, categories = get_brand_categories(brands)
categories = list(categories)

cat_alignment_df = pd.DataFrame(index=categories)
    
for i in range(len(all_models)):
    
    curr_title = titles[i]
    curr_vocab = all_vocabs[i]
    curr_model = all_models[i]
    print(f"MODEL: {curr_title}")
    
    he_she_axis = get_gender_axis(curr_model)
#     print(f"Getting word vectors")
#     vecs = get_word_vectors(curr_model, all_vocabs[i], brands)
    
#     alignment = get_simple_alignment(vecs, he_she_axis)
#     print(alignment.shape)
#     alignment_df[f'{curr_title}_alignment'] = alignment
    
    category_vec_map = {}
    category_vecs = []
    for category in categories:
        new_vec = get_category_vectors(curr_model, curr_vocab, category)
#         category_vec_map[category] = new_vec
        category_vecs.append(new_vec)
    category_vecs = np.array(category_vecs)
    category_alignment = get_simple_alignment(category_vecs, he_she_axis)
    print(category_alignment.shape)
    cat_alignment_df[f'{curr_title}_alignment'] = np.round(category_alignment, 3)
    cat_alignment_df[f'{curr_title}_gender'] = cat_alignment_df[f'{curr_title}_alignment'].apply(lambda x: 1 if x >=0 else 0)

    
#     residual_vecs = get_residual_vecs(vecs, category_vec_map, brand_category_mapping, brands)
    
#     resid_alignment = get_simple_alignment(residual_vecs, he_she_axis)
#     alignment_df[f'{curr_title}_resid_alignment'] = resid_alignment
    
# alignment_df.to_csv('../Data/all_models_alignment_11_06.csv')
    
cat_alignment_df.to_csv('../Data/all_models_category_alignment_v3.csv')
    
    
    

    

MODEL: Word2Vec
(72, 1)
MODEL: Wikipedia
athleticwear
NEW VEC
(300,)
(72, 1)
MODEL: Glove
athleticwear
NEW VEC
(300,)
(72, 1)


In [24]:

print(pearsonr(cat_alignment_df['Glove_alignment'], cat_alignment_df['Word2Vec_alignment']))
print(pearsonr(cat_alignment_df['Wikipedia_alignment'], cat_alignment_df['Word2Vec_alignment']))
print(pearsonr(cat_alignment_df['Glove_alignment'], cat_alignment_df['Wikipedia_alignment']))

NameError: name 'pearsonr' is not defined

# Pre vs post residualization, % of brands classified as female

In [25]:
alignment_df = pd.read_csv('../Data/all_models_alignment_27_08_v3.csv')
alignment_df['category'] = alignment_df['brand'].apply(lambda x: brand_category_mapping[x])
# gender_prop_df = pd.DataFrame({})
gender_prop_dict= {}
for i in range(len(all_models)):
    curr_title = titles[i]
    split_1 = 72//4
    split_2 = 3*split_1
    new_df = cat_alignment_df.sort_values(by=f'{curr_title}_alignment')
    male_cats = new_df.index[:split_1]
    female_cats = new_df.index[split_2:]
    neutral_cats = new_df.index[split_1:split_2]
    
    
    male_df = alignment_df[alignment_df['category'].isin(male_cats)]
    female_df = alignment_df[alignment_df['category'].isin(female_cats)]
    neutral_df = alignment_df[alignment_df['category'].isin(neutral_cats)]
    
    overall_female_proportion =  alignment_df[alignment_df[f"{curr_title}_alignment"] > 0].shape[0] / alignment_df.shape[0]
    
    male_female_proportion = male_df[male_df[f"{curr_title}_alignment"] > 0].shape[0] / male_df.shape[0]
    female_female_proportion = female_df[female_df[f"{curr_title}_alignment"] > 0].shape[0] / female_df.shape[0]
    neutral_female_proportion = neutral_df[neutral_df[f"{curr_title}_alignment"] > 0].shape[0] / neutral_df.shape[0]

    print(f"{curr_title} female percentage before residualization (male): {male_female_proportion}")
    print(f"{curr_title} female percentage before residualization (female): {female_female_proportion}")
    print(f"{curr_title} female percentage before residualization (neutral): {neutral_female_proportion}")

    resid_overall_female_proportion =  alignment_df[alignment_df[f"{curr_title}_resid_alignment"] > 0].shape[0] / alignment_df.shape[0]

    resid_male_female_proportion = male_df[male_df[f"{curr_title}_resid_alignment"] > 0].shape[0] / male_df.shape[0]
    resid_female_female_proportion = female_df[female_df[f"{curr_title}_resid_alignment"] > 0].shape[0] / female_df.shape[0]
    resid_neutral_female_proportion = neutral_df[neutral_df[f"{curr_title}_resid_alignment"] > 0].shape[0] / neutral_df.shape[0]

    print(f"{curr_title} female percentage after residualization (male): {resid_male_female_proportion}")
    print(f"{curr_title} female percentage after residualization (female): {resid_female_female_proportion}")
    print(f"{curr_title} female percentage after residualization (neutral): {resid_neutral_female_proportion}")
    gender_prop_dict[curr_title] = {'Overall': overall_female_proportion,
                                    'Male': male_female_proportion,
                                    'Female': female_female_proportion,
                                    'Neutral': neutral_female_proportion,
                                    'Residual Overall': resid_overall_female_proportion,
                                   'Residual Male': resid_male_female_proportion,
                                    'Residual Female': resid_female_female_proportion,
                                    'Residual Neutral': resid_neutral_female_proportion}

gender_differences = pd.DataFrame(gender_prop_dict).transpose()


    
    

Word2Vec female percentage before residualization (male): 0.6901408450704225
Word2Vec female percentage before residualization (female): 0.8371040723981901
Word2Vec female percentage before residualization (neutral): 0.7447154471544716
Word2Vec female percentage after residualization (male): 0.7300469483568075
Word2Vec female percentage after residualization (female): 0.2398190045248869
Word2Vec female percentage after residualization (neutral): 0.3853658536585366
Wikipedia female percentage before residualization (male): 0.0030120481927710845
Wikipedia female percentage before residualization (female): 0.023872679045092837
Wikipedia female percentage before residualization (neutral): 0.02531645569620253
Wikipedia female percentage after residualization (male): 0.7108433734939759
Wikipedia female percentage after residualization (female): 0.17771883289124668
Wikipedia female percentage after residualization (neutral): 0.43037974683544306
Glove female percentage before residualization (

In [26]:
gender_differences['Female Diff'] = gender_differences['Female'] - gender_differences['Residual Female']
gender_differences['Male Diff'] = gender_differences['Male'] - gender_differences['Residual Male']
gender_differences['Neutral Diff'] = gender_differences['Neutral'] - gender_differences['Residual Neutral']
gender_differences.round(3).to_csv('effect of residualization on female alignment v3.csv')

# Word2Phrase version

In [110]:
all_models =  [w2v_model, ft_wiki_model, glove_model] # 
all_vocabs =  [w2v_vocab, ft_wiki_vocab, glove_vocab] #w2v_vocab,
titles = ['Word2Vec', 'Wikipedia', 'Glove']
alignment_df = pd.read_csv('../Data/alignment_scores.csv')
brands = alignment_df['brand'].tolist()


brand_category_mapping, categories = get_brand_categories(brands, concise=True)
categories = list(categories)

    
for i in range(1):
    
    curr_title = titles[i]
    curr_vocab = all_vocabs[i]
    curr_model = all_models[i]
    print(f"MODEL: {curr_title}")
    
    he_she_axis = get_gender_axis(curr_model)
    print(f"Getting word vectors")
    vecs = get_word_vectors(curr_model, curr_vocab, brands)
    
    alignment = get_simple_alignment(vecs, he_she_axis)
    print(alignment.shape)
    alignment_df[f'{curr_title}_alignment'] = alignment
    
    category_vec_map = {}
    for category in categories:
#         print(category)
        new_vec = get_category_vectors(curr_model, curr_vocab, category)
        category_vec_map[category] = new_vec
    residual_vecs = get_residual_vecs(vecs, category_vec_map, brand_category_mapping, brands)
    
    resid_alignment = get_simple_alignment(residual_vecs, he_she_axis)
    alignment_df[f'{curr_title}_resid_alignment'] = resid_alignment
    
alignment_df.to_csv('../Data/all_models_alignment_27_08_concise.csv')
    
    

MODEL: Word2Vec
Getting word vectors



NUM CAPITALIZED: 1298/1471 = 0.882
(1471, 300)
(1471, 1)
skin_care_products
products_or_services
multipurpose_cleaner
security_system
fire_safety
automotive_batteries
web_design
hosting_service_providers
insurance_agency
insurance_brokers
television_network
television_stations
production_company
beauty_shop
products_or_services
toy_store
internet_service_providers
sporting_goods_store
cable_television
vehicle_sales
vehicle_leasing
fishing_tackle
RESIDUAL VECTORS SHAPE
(1471, 300)


In [111]:

cat_alignment_df = pd.DataFrame(index=categories)
    
for i in range(1):
    
    curr_title = titles[i]
    curr_vocab = all_vocabs[i]
    curr_model = all_models[i]
    print(f"MODEL: {curr_title}")
    
    he_she_axis = get_gender_axis(curr_model)
#     print(f"Getting word vectors")
#     vecs = get_word_vectors(curr_model, all_vocabs[i], brands)
    
#     alignment = get_simple_alignment(vecs, he_she_axis)
#     print(alignment.shape)
#     alignment_df[f'{curr_title}_alignment'] = alignment
    
    category_vec_map = {}
    category_vecs = []
    for category in categories:
        new_vec = get_category_vectors(curr_model, curr_vocab, category)
#         category_vec_map[category] = new_vec
        category_vecs.append(new_vec)
    category_vecs = np.array(category_vecs)
    category_alignment = get_simple_alignment(category_vecs, he_she_axis)
    print(category_alignment.shape)
    cat_alignment_df[f'{curr_title}_alignment'] = np.round(category_alignment, 3)
    cat_alignment_df[f'{curr_title}_gender'] = cat_alignment_df[f'{curr_title}_alignment'].apply(lambda x: 1 if x >=0 else 0)

    
#     residual_vecs = get_residual_vecs(vecs, category_vec_map, brand_category_mapping, brands)
    
#     resid_alignment = get_simple_alignment(residual_vecs, he_she_axis)
#     alignment_df[f'{curr_title}_resid_alignment'] = resid_alignment
    
# alignment_df.to_csv('../Data/all_models_alignment_11_06.csv')
    
cat_alignment_df.to_csv('../Data/all_models_category_alignment_concise.csv')

MODEL: Word2Vec
skin_care_products
products_or_services
multipurpose_cleaner
security_system
fire_safety
automotive_batteries
web_design
hosting_service_providers
insurance_agency
insurance_brokers
television_network
television_stations
production_company
beauty_shop
products_or_services
toy_store
internet_service_providers
sporting_goods_store
cable_television
vehicle_sales
vehicle_leasing
fishing_tackle
(72, 1)


In [115]:
alignment_df = pd.read_csv('../Data/all_models_alignment_27_08_concise.csv')
alignment_df['category'] = alignment_df['brand'].apply(lambda x: brand_category_mapping[x])
# gender_prop_df = pd.DataFrame({})
gender_prop_dict= {}
for i in range(1):
    curr_title = titles[i]
    split_1 = 72//4
    split_2 = 3*split_1
    new_df = cat_alignment_df.sort_values(by=f'{curr_title}_alignment')
    male_cats = new_df.index[:split_1]
    female_cats = new_df.index[split_2:]
    neutral_cats = new_df.index[split_1:split_2]
    male_df = alignment_df[alignment_df['category'].isin(male_cats)]
    female_df = alignment_df[alignment_df['category'].isin(female_cats)]
    neutral_df = alignment_df[alignment_df['category'].isin(neutral_cats)]
    
    overall_female_proportion =  alignment_df[alignment_df[f"{curr_title}_alignment"] > 0].shape[0] / alignment_df.shape[0]
    
    male_female_proportion = male_df[male_df[f"{curr_title}_alignment"] > 0].shape[0] / male_df.shape[0]
    female_female_proportion = female_df[female_df[f"{curr_title}_alignment"] > 0].shape[0] / female_df.shape[0]
    neutral_female_proportion = neutral_df[neutral_df[f"{curr_title}_alignment"] > 0].shape[0] / neutral_df.shape[0]

    print(f"{curr_title} female percentage before residualization (male): {male_female_proportion}")
    print(f"{curr_title} female percentage before residualization (female): {female_female_proportion}")
    print(f"{curr_title} female percentage before residualization (neutral): {neutral_female_proportion}")

    resid_overall_female_proportion =  alignment_df[alignment_df[f"{curr_title}_resid_alignment"] > 0].shape[0] / alignment_df.shape[0]

    resid_male_female_proportion = male_df[male_df[f"{curr_title}_resid_alignment"] > 0].shape[0] / male_df.shape[0]
    resid_female_female_proportion = female_df[female_df[f"{curr_title}_resid_alignment"] > 0].shape[0] / female_df.shape[0]
    resid_neutral_female_proportion = neutral_df[neutral_df[f"{curr_title}_resid_alignment"] > 0].shape[0] / neutral_df.shape[0]

    print(f"{curr_title} female percentage after residualization (male): {resid_male_female_proportion}")
    print(f"{curr_title} female percentage after residualization (female): {resid_female_female_proportion}")
    print(f"{curr_title} female percentage after residualization (neutral): {resid_neutral_female_proportion}")
    gender_prop_dict[curr_title] = {'Overall': overall_female_proportion,
                                    'Male': male_female_proportion,
                                    'Female': female_female_proportion,
                                    'Neutral': neutral_female_proportion,
                                    'Residual Overall': resid_overall_female_proportion,
                                   'Residual Male': resid_male_female_proportion,
                                    'Residual Female': resid_female_female_proportion,
                                    'Residual Neutral': resid_neutral_female_proportion}

gender_differences = pd.DataFrame(gender_prop_dict).transpose()
gender_differences['Female Diff'] = gender_differences['Female'] - gender_differences['Residual Female']
gender_differences['Male Diff'] = gender_differences['Male'] - gender_differences['Residual Male']
gender_differences['Neutral Diff'] = gender_differences['Neutral'] - gender_differences['Residual Neutral']
gender_differences.round(3).to_csv('concise effect of residualization on female alignment.csv')

    
    

Word2Vec female percentage before residualization (male): 0.6839285714285714
Word2Vec female percentage before residualization (female): 0.8105263157894737
Word2Vec female percentage before residualization (neutral): 0.7587859424920128
Word2Vec female percentage after residualization (male): 0.7
Word2Vec female percentage after residualization (female): 0.24912280701754386
Word2Vec female percentage after residualization (neutral): 0.402555910543131


In [116]:
gender_differences.round(3)

,Female,Male,Neutral,Overall,Residual Female,Residual Male,Residual Neutral,Residual Overall,Female Diff,Male Diff,Neutral Diff
Word2Vec,0.811,0.684,0.759,0.74,0.249,0.7,0.403,0.486,0.561,-0.016,0.356


# CATEGORY PLOTS

In [108]:
df_brands = pd.read_csv('../Data/all_models_alignment_27_08.csv')
df_brands = df_brands.replace(0, np.nan).dropna()
df_brands = df_brands[df_brands.columns[2:]]
df_brands['category'] = df_brands['brand'].apply(lambda x: brand_category_mapping[x])
count_df = df_brands.groupby(by='category').count()
frequent_categories = count_df[count_df['brand'] > 20].index.tolist()
shortened_df = df_brands[df_brands['category'].isin(frequent_categories)]
prefixes = ['Word2Vec', 'Wikipedia', 'Glove']
for resid in ['', '_resid']:
    for pref in prefixes:
        shortened_df[f'gender_{pref}{resid}'] = shortened_df[f'{pref}{resid}_alignment'].apply(lambda x: 1 if x >=0 else 0)
category_counts = shortened_df.groupby(by='category').sum()
for resid in ['', '_resid']:
    for pref in prefixes:
        category_counts[f'female_{pref}{resid}'] = category_counts[f'gender_{pref}{resid}'] / count_df['brand']

category_counts

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,alignment,Word2Vec_alignment,Word2Vec_resid_alignment,Wikipedia_alignment,Wikipedia_resid_alignment,Glove_alignment,Glove_resid_alignment,gender_Word2Vec,gender_Wikipedia,gender_Glove,gender_Word2Vec_resid,gender_Wikipedia_resid,gender_Glove_resid,female_Word2Vec,female_Wikipedia,female_Glove,female_Word2Vec_resid,female_Wikipedia_resid,female_Glove_resid
category,,,,,,,,,,,,,,,,,,,
advertising marketing printing services,1.202050,1.202050,-0.844806,-4.543579,-1.446578,-0.392175,0.929515,27,1,14,14,8,24,0.729730,0.027027,0.378378,0.378378,0.216216,0.648649
bedding linens,2.313414,2.313414,-1.944253,-3.177260,-0.277719,0.544153,-2.685862,22,0,14,4,11,3,0.846154,0.000000,0.538462,0.153846,0.423077,0.115385
beer ale,1.961496,1.961496,3.540882,-7.934983,5.315436,-1.041373,1.741575,39,0,21,47,57,40,0.684211,0.000000,0.368421,0.824561,1.000000,0.701754
casual dining restaurants,1.655916,1.655916,-0.185471,-5.222551,-1.366048,-0.542295,-1.823177,28,0,20,18,12,5,0.717949,0.000000,0.512821,0.461538,0.307692,0.128205
cookies crackers,2.473957,2.473957,-0.961461,-3.189858,-1.622384,1.008063,-1.899345,24,2,21,11,4,5,0.827586,0.068966,0.724138,0.379310,0.137931,0.172414
credit cards,1.329122,1.329122,1.492480,-3.090280,1.057648,-0.114361,0.588543,24,1,12,24,21,20,0.888889,0.037037,0.444444,0.888889,0.777778,0.740741
dinnerware glassware pottery,2.039605,2.039605,-2.580129,-3.723958,-1.389092,0.993246,-0.842025,23,1,20,1,5,9,0.793103,0.034483,0.689655,0.034483,0.172414,0.310345
fishing tackle equipment,1.219839,1.219839,3.668864,-4.547282,1.290484,-0.784948,2.475317,25,0,11,30,22,31,0.781250,0.000000,0.343750,0.937500,0.687500,0.968750
handbags,3.402585,3.402585,-2.003226,-2.339662,-2.339755,2.027201,-1.431812,22,1,20,4,1,4,0.880000,0.040000,0.800000,0.160000,0.040000,0.160000


In [86]:
prefixes = ['Word2Vec', 'Wikipedia', 'Glove']
for resid in ['', '_resid']:
    for pref in prefixes:
        old_order = category_counts.sort_values(by=f'female_{pref}{resid}')  
        cat_ind = old_order.index
        indices = [i for i in range(len(cat_ind))]
        sub_m = (1 - old_order[f'female_{pref}{resid}']).tolist()
        sub_f = old_order[f'female_{pref}{resid}'].tolist()
        sub_c = old_order.index
        barWidth = 0.5
        plt.bar(indices, sub_m, color='#0b6304', width=barWidth, label="Male")
        plt.bar(indices, sub_f, bottom=sub_m, color='#f78725', width=barWidth, label="Female")
        plt.xticks(indices, sub_c, rotation='vertical', fontsize=8)
        plt.xlabel("Categories")
        plt.ylabel("Proportion Female (%)")
        plt.title(f"Gender Distribution across {pref}{resid} Categories")
        plt.legend()
        plt.axhline(y=0.5)
        plt.rcParams["figure.figsize"] = [10, 5]
        plt.gcf().subplots_adjust(bottom=0.50)
        plt.savefig(f'../Plots/category_distribution_{pref}{resid}_06_11.png', dpi=150)
        plt.clf()


<Figure size 720x360 with 0 Axes>

In [137]:
df_brands = pd.read_csv('../Data/all_models_alignment_11_06.csv')
df_brands = df_brands.replace(0, np.nan).dropna()
df_brands = df_brands[df_brands.columns[2:]]
df_brands['category'] = df_brands['brand'].apply(lambda x: brand_category_mapping[x])
count_df_mean = df_brands.groupby(by='category').mean()
count_df_mean = count_df_mean[count_df_mean.columns[1:]]
count_df_std = df_brands.groupby(by='category').std()
count_df_std = count_df_std[count_df_std.columns[1:]]


In [138]:
new_names = [column + '_std' for column in count_df_std.columns.tolist()]
count_df_std.columns=new_names

In [139]:
count_df_std

,Word2Vec_alignment_std,Word2Vec_resid_alignment_std,Wikipedia_alignment_std,Wikipedia_resid_alignment_std,Glove_alignment_std,Glove_resid_alignment_std
category,,,,,,
advertising marketing printing services,0.060172,0.055724,0.057506,0.052569,0.060726,0.047342
amusement water theme parks,0.001815,0.001040,0.016009,0.046105,0.029941,0.030740
artificial sweeteners,0.076253,0.074961,0.065423,0.067716,0.082321,0.067175
automotive batteries,0.031175,0.028754,0.002415,0.026469,0.020720,0.008582
beauty shops,0.014396,0.065153,0.012186,0.046406,0.078933,0.057100
bedding linens,0.090838,0.068960,0.065829,0.056502,0.092243,0.060221
beer ale,0.073501,0.057348,0.049699,0.032014,0.077597,0.057056
cable television providers systems,NaN,NaN,NaN,NaN,NaN,NaN
cameras,0.052936,0.039324,0.063976,0.059419,0.039087,0.030917


In [141]:
mean_std = pd.DataFrame(count_df[count_df.columns[:1]])
mean_std = mean_std.rename(columns={'brand': 'Sample Size'})
for i in range(len(count_df_mean.columns)):
    mean_col = count_df_mean.columns[i]
    std_col = count_df_std.columns[i]
    mean_std[mean_col] = count_df_mean[mean_col]
    mean_std[std_col] = count_df_std[std_col]

In [143]:
mean_std.to_csv('../Data/average_std_categories.csv')

In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('../Data/all_models_alignment_11_06.csv').replace(0, np.nan).dropna()
web = pd.read_csv('../Data/brand_website_data_jan.csv')

In [11]:
web.shape

(654, 11)

In [13]:
web[web['Name'].isin(df['brand'])].to_csv('../Data/filtered_web_brands.csv')